**Overtone Extraction**

This script extracts overtone magnitudes and phases with previously extracted pitch and pitch increment trajectories.  
Overtones are extracted by per-frame division of source x by complex sinusoidal taking pitch and pitch derivative into account.

In [ ]:
%load_ext autoreload
%autoreload 2

#hsvs includes
from hsvs.tools import dataset, framed_audio, magnitude, extract_overtones
import hsvs

# 3rd party dependencies
from matplotlib import pyplot as plt
import numpy as np
import soundfile
from tqdm import tqdm

In [ ]:
no_vocalset = True # set to true if vocalset directory is not set. If True, uses data/input source samples
vowel  = 'a'  # [a,e,i,o,u] or [a,i,o] if no_vocalset == True
singer = 'f4' # [f1 - f9, m1-m11] or [f1, f4, f5, f6, f8, m1, m2, m4, m5, m8] if no_vocalset == True

num_threads_extraction = 6 # set to number of threads to speed up extractio a bit

truncate_audible = True
num_overtones_extracted = 60 # overtones 
num_overtones_stored    = 40

# wav and json file path 
singer_vowel_dir = singer + '_' + vowel
data_path   = os.path.abspath(os.path.join(os.path.dirname(hsvs.__file__), os.pardir, 'data'))

if(no_vocalset):
    source_file = os.path.join(data_path, 'input', singer + '_scales_c_slow_forte_' + vowel + '.wav')
else:
    source_file = dataset.get_sample('scales', 'slow_forte', vowel, singer)[0]

json_file   = os.path.join(data_path, 'results', singer_vowel_dir, 'audio.json' )
out_file_harm = os.path.join(data_path, 'results', singer_vowel_dir, 'audio', 'harmonic.wav' )
out_file_org  = os.path.join(data_path, 'results', singer_vowel_dir, 'audio', 'original.wav' )

print(source_file)

In [ ]:
# loading the stored audio json file explicitely with the audio wave file to prevent portability issues.
audio = framed_audio.FramedAudio.from_json(json_file, wav_replacement=source_file)

#run overtone extraction, pitch and pitch-inc are assumed to be stored as trajectories in the audio object
overtones = extract_overtones.extract_overtones_from_audio(audio, num_overtones = num_overtones_extracted, num_threads=num_threads_extraction)

# convert to decibel and normalize
overtonesDB  = 20. * np.log10(np.abs(overtones))
overtonesDB -= np.max(overtonesDB)

# store in json
audio.store_trajectory('overtones', overtonesDB[:, 0:num_overtones_stored])

In [ ]:
# plot first 10 overtone trajectories
plots = plt.plot(audio.get_time(), overtonesDB[:,0:9])

**Reconstruction**

The following cell reconstructs the audio from the pitch and pitch increment trajectories as well as the overtone data.  
Reconstruction is performed per-frame with an overlapp add method and hanning windowing

In [ ]:
# synthesize audio
pitch     = audio.get_trajectory('pitch')
pitch_inc = audio.get_trajectory('pitch-inc')

x = np.zeros(audio.get_num_frames() * audio.hop_size + audio.block_size)
window = 2 * np.hanning(audio.block_size)
for i in tqdm(range(audio.get_num_frames())):
    frame = extract_overtones.synthesize_overtones(overtones[i,:], pitch[i], pitch_inc[i], audio.fs, audio.block_size)

    x0 =  i * audio.hop_size
    x1 = x0 + audio.block_size

    overlap_factor = 2 * audio.hop_size / audio.block_size 
    x[x0:x1] += frame * window * overlap_factor



In [ ]:
#store results
audio.save_json(json_file=json_file)

# create dirs if they don't exist yet
audio_path = os.path.dirname(out_file_harm)
if not os.path.exists(audio_path):
            os.makedirs(audio_path)
            
org = audio.get_raw()

# if true, truncates the audio signals to the same length as the synthesized signals
if(truncate_audible):
    onset, offset = magnitude.get_onset_offset(audio)
    onset_audio   = (onset  + 50) * audio.hop_size 
    offset_audio  = (offset - 50) * audio.hop_size

    x_truncated = x[onset_audio:offset_audio]
    org_truncated = org[onset_audio:offset_audio]

else:
    x_truncated = x
    org_truncated = org



# store synthesized and original audio 
soundfile.write(out_file_harm, x_truncated  / (max(abs(x_truncated))), audio.fs)
soundfile.write(out_file_org, org_truncated / (max(abs(org_truncated))), audio.fs)


In [ ]:
# resetting the kernel to flush unused memory
# notification beep.

#import numpy as np
#import sounddevice
#beep = np.sin(np.linspace(0, 1000*np.pi,22050))
#sounddevice.play(beep, 44100, blocking=True)

#%reset -f
#exit()